# Library

Importamos librerias

In [1]:
import pyRofex as pyRofex
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as datetime

# plotly dash

# Import Data

## Log in

Hacemos el Login para poder traer datos de la API de Rofex

In [2]:
pyRofex.initialize(user="luciomassimi3968",
                   password="feulrX6%",
                   account="REM3968",
                   environment=pyRofex.Environment.REMARKET)

## Spot

Seteamos los parametros para traer datos del spot.

In [3]:
tickers_spot = ['GGAL.BA','PAMP.BA','YPFD.BA','ARS=X']

start = datetime.datetime(2022,1,1) # Starting date in format (year,month,day)
end = datetime.datetime.today() # Ending date

In [4]:
# spot_df = pd.DataFrame()
# for ticker in tickers_spot:
#     """
#     This will make a dataframe with the "Close" of each of our stocks
#     """
#     spot_df[ticker] = yf.Ticker(ticker).history(start=end,end=end)['Close']

# spot_df.columns =['GGAL','PAMP','YPFD','DLR']
# spot_df.tail()

In [5]:
spot_df = pd.DataFrame()
for ticker in tickers_spot:
    """
    This will make a dataframe with the "Close" of each of our stocks
    """
    spot_df[ticker] = yf.Ticker(ticker).history(start=start,end=end)['Close']

spot_df.columns =['GGAL','PAMP','YPFD','DLR']
spot_df.tail()

,GGAL,PAMP,YPFD,DLR
Date,,,,
2022-01-19,191.050003,160.149994,866.299988,104.122375
2022-01-20,195.699997,164.850006,877.349976,104.201080
2022-01-21,189.399994,166.250000,851.650024,104.261841
2022-01-24,181.550003,163.800003,826.200012,104.350235
2022-01-25,183.399994,165.250000,831.049988,104.589996


In [6]:
# spot.T.iloc[:,-1]

In [7]:
spot_last= spot_df.iloc[-1]
spot_last

GGAL    183.399994
PAMP    165.250000
YPFD    831.049988
DLR     104.589996
Name: 2022-01-25 00:00:00, dtype: float64

## Futures

### Import GGAL, PAMP, YFD, DLR/FUT

Seteamos los tickers de los activos que queremos traer, en este caso traemos los futuros de GGAL, PAMP, YFD, y todos los futuros DLR/FUT de 2022

In [8]:
futuros_dolar=['DLR/ENE22','DLR/FEB22','DLR/MAR22','DLR/ABR22','DLR/MAY22','DLR/JUN22','DLR/JUL22','DLR/AGO22','DLR/SEP22','DLR/OCT22','DLR/NOV22','DLR/DIC22']
futuro_ggal=['GGAL/FEB22']
futuro_pamp=['PAMP/FEB22']
futuro_ypfd=['YPFD/FEB22']

tickers= futuros_dolar + futuro_ggal + futuro_pamp + futuro_ypfd
depth=5

Usamos la funcion 'get_market_data' para traer la data de la rest API de rofex, y la almacenamos en un diccionario para luego poder extraer los datos que necesitemos

In [9]:
resp_dict = {}

for instrument in tickers:
    resp_dict[f"{instrument}"] = pyRofex.get_market_data(ticker = instrument, depth = depth)
    
resp_dict

{'DLR/ENE22': {'status': 'OK',
  'marketData': {'OF': [],
   'CL': {'price': 105.2, 'size': None, 'date': 1642982400000},
   'IV': None,
   'BI': [{'price': 105.13, 'size': 65},
    {'price': 104.38, 'size': 200},
    {'price': 103.7, 'size': 66},
    {'price': 103.63, 'size': 200}],
   'NV': 2076000,
   'EV': 219336710.0,
   'OP': 105.2,
   'SE': {'price': 116.0, 'size': None, 'date': 1630281600000},
   'LA': {'price': 105.13, 'size': 1, 'date': 1643122301606},
   'HI': 106.5,
   'TV': 2076.0,
   'LO': 105.1,
   'OI': {'price': None, 'size': 103485, 'date': 1630195200000}},
  'depth': 5,
  'aggregated': True},
 'DLR/FEB22': {'status': 'OK',
  'marketData': {'OF': [{'price': 108.73, 'size': 99},
    {'price': 108.93, 'size': 100},
    {'price': 109.56, 'size': 100},
    {'price': 109.72, 'size': 100},
    {'price': 110.51, 'size': 100}],
   'CL': {'price': 108.7, 'size': None, 'date': 1642982400000},
   'IV': None,
   'BI': [{'price': 108.65, 'size': 100},
    {'price': 108.42, 'size':

### BID - OFFER per instrument

Del diccionario de datos que creamos a partir del query, extraemos los precios del BID y OFFER de los distintos activos. A veces como la liquidez es muy limitada, no hay puntas en el BID ni en el OFFER. Por esto tambien agregue un nested "try - Except" por si es totalmente necesario traer algun dato. Esto a la hora de tradear o correr el bot no deberia hacerse, y solo se deberian tomar los datos de BID y OFFER. En este caso lo hago para que se vean mas datos cuando se corre la notebook.

In [10]:
# #BID
# bid={}
# for instrument in tickers:
#     try:
#         bid[instrument] = resp_dict[instrument]['marketData']['BI'][0]['price']
#     except:
#         print(f"{instrument} has no BID")
#         bid[instrument] = np.nan

# bid

In [11]:
#BID
bid={}
for instrument in tickers:
    #Trato de traer los valores del precio
    try:
        bid[instrument] = resp_dict[instrument]['marketData']['BI'][0]['price']
    
    #si no trae valores de precio y necesito probar algo de las funciones que siguen, puedo tratar de importar los
    #datos del cierre o del ultimo precio
    except:
#         try:
#             print(f"{instrument} has no BID")
#             bid[instrument] = resp_dict[instrument]['marketData']['LA'][0]['price']
            
#         except:
#             try:
#                 bid[instrument] = resp_dict[instrument]['marketData']['CL']['price']
        
#             except:
                print(f"{instrument} has no BID")
                bid[instrument] = np.nan

bid

DLR/JUL22 has no BID
DLR/AGO22 has no BID
DLR/SEP22 has no BID
DLR/OCT22 has no BID
DLR/NOV22 has no BID
DLR/DIC22 has no BID
GGAL/FEB22 has no BID
YPFD/FEB22 has no BID


{'DLR/ENE22': 105.13,
 'DLR/FEB22': 108.65,
 'DLR/MAR22': 114.0,
 'DLR/ABR22': 118.6,
 'DLR/MAY22': 123.6,
 'DLR/JUN22': 129.2,
 'DLR/JUL22': nan,
 'DLR/AGO22': nan,
 'DLR/SEP22': nan,
 'DLR/OCT22': nan,
 'DLR/NOV22': nan,
 'DLR/DIC22': nan,
 'GGAL/FEB22': nan,
 'PAMP/FEB22': 168.6,
 'YPFD/FEB22': nan}

In [12]:
# #OFFER
# offer={}
# for instrument in tickers:
#     try:
#         offer[instrument] = resp_dict[instrument]['marketData']['OF'][0]['price']
#     except:
#         print(f"{instrument} has no OFFER")
#         offer[instrument] = np.nan

# offer

In [13]:
#OFFER
offer={}
for instrument in tickers:
    #Trato de traer los valores del precio
    
    try:
        offer[instrument] = resp_dict[instrument]['marketData']['OF'][0]['price']
        
    except:
        
        #si no trae valores de precio y necesito probar algo de las funciones que siguen, puedo tratar de importar los
        #datos del cierre o del ultimo precio
#         try:
#             print(f"{instrument} has no OFFER")
#             offer[instrument] = resp_dict[instrument]['marketData']['LA'][0]['price']
            
#         except:
#             try:
#                 offer[instrument] = resp_dict[instrument]['marketData']['CL']['price']
        
#             except:
                print(f"{instrument} has no OFFER")
                offer[instrument] = np.nan

offer

DLR/ENE22 has no OFFER
DLR/JUL22 has no OFFER
DLR/AGO22 has no OFFER
DLR/SEP22 has no OFFER
DLR/OCT22 has no OFFER
DLR/NOV22 has no OFFER
DLR/DIC22 has no OFFER
GGAL/FEB22 has no OFFER


{'DLR/ENE22': nan,
 'DLR/FEB22': 108.73,
 'DLR/MAR22': 114.15,
 'DLR/ABR22': 118.95,
 'DLR/MAY22': 124.2,
 'DLR/JUN22': 129.25,
 'DLR/JUL22': nan,
 'DLR/AGO22': nan,
 'DLR/SEP22': nan,
 'DLR/OCT22': nan,
 'DLR/NOV22': nan,
 'DLR/DIC22': nan,
 'GGAL/FEB22': nan,
 'PAMP/FEB22': 169.4,
 'YPFD/FEB22': 902.35}

Creo el dataframe con los valores de BID y OFFER para cada uno de los activos que pido

In [14]:
fut = pd.DataFrame({"bid":bid,"offer":offer})
fut

,bid,offer
DLR/ENE22,105.13,NaN
DLR/FEB22,108.65,108.73
DLR/MAR22,114.00,114.15
DLR/ABR22,118.60,118.95
DLR/MAY22,123.60,124.20
DLR/JUN22,129.20,129.25
DLR/JUL22,NaN,NaN
DLR/AGO22,NaN,NaN
DLR/SEP22,NaN,NaN
DLR/OCT22,NaN,NaN


Si es necesario, se puede agregar la comision de compra y venta para el futuro para saber cuales serian las tasas reales en caso de operar
(lo mismo se deberia hacer para el spot)

In [15]:
#comision
# comision = .005

# fut['offer']=fut['offer']*(1+comision)
# fut['bid']=fut['bid']*(1-comision)
# fut

# Modify Data

## Defining functions: 

### Tasas

Creamos distintas funciones para obtener la tasa. La tasa va a variar dependiendo si usamos tasa de compounding continuo(que es lo que se suele usar en derivados), si se calcula TNA, TEA, etc. Para esto se crea una funcion que engloba a todas las otras funciones, y cambiando la variable "TIPO" vamos a poder elegir entre una u otra. Tambien se puede seleccionar el input temporal como dia o mes de ser necesario

In [16]:
spot_v=100
future_v=105
dias=30
mes=1

Tasa de compounding continuo, gralmente usada para derivados.

In [17]:
def tasa_continua_mes(spot,future,t=1):
    tasa=np.log(future/spot) / (t/12)
    return(tasa)

tasa_continua_mes(spot_v,future_v,mes)

0.5854819700331846

In [18]:
def tasa_continua_dias(spot,future,t=30):
    tasa=np.log(future/spot) / (t/365)
    return(tasa)

tasa_continua_dias(spot_v,future_v,dias)

0.5936136640614232

Tasa Nocional Anual, el equivalente seria la tasa que se ve en los plazo fijos

In [19]:
def tasa_tna_mes(spot,future,t=1):
    tasa=((future/spot)-1)*(12/t)
    return(tasa)

tasa_tna_mes(spot_v,future_v,mes)

0.6000000000000005

In [20]:
def tasa_tna_dias(spot,future,t=30):
    tasa=((future/spot)-1)*(365/t)
    return(tasa)

tasa_tna_dias(spot_v,future_v,dias)

0.6083333333333338

Tasa Efectiva Anual, es la tasa 'REAL' que estariamos percibiendo en el ano. La diferencia con la nocional, es que la nocional no toma en cuenta el compounding y en esta si se tiene en cuenta. 
Por ej, si ponemos un plazo fijo con TNA 30% a un ano, 100>130. Pero si en vez de ponerla a un ano se pone a la misma tasa TNA 30% pero cada mes, al final del ano se va a tener >130 ya que cada mes se va a tener un poco de retorno y ese retorno tambien se va a colocar. Por eso la TEA > TNA

In [21]:
def tasa_tea_mes(spot,future,t=30):
    tasa=((future/spot)**(12/t))-1
    return(tasa)

tasa_tea_mes(spot_v,future_v,mes)

0.7958563260221301

In [22]:
def tasa_tea_dias(spot,future,t=30):
    tasa=((future/spot)**(365/t))-1
    return(tasa)

tasa_tea_dias(spot_v,future_v,dias)

0.8105192164554333

La tasa directa es el % real sin anualizar

In [23]:
def tasa_directa(spot,future):
    tasa=(future/spot)-1
    return(tasa)

tasa_directa(spot_v,future_v)

0.050000000000000044

In [24]:
spot_v, future_v

(100, 105)

Esta funcion toma todas las otras funciones calculadoras de tasa y las junta en una sola para que sea mas simple seleccionar la tasa que se quiere usar. En vez de cambiar todas las funciones, se puede poner una variable como 'tipo' antes de correr las funciones y seleccionar el tipo de tasa que se quiere usar de esa manera. Cumpliria la funcion de 'switch'

In [25]:
# def tipo_tasa(tipo,period,spot,future,t=1):
    
#     if tipo.lower()== "continua":
        
#         if period.lower()=="mes":
#             return(tasa_continua_mes(spot,future,t))
        
#         elif period.lower()=="dias":
#             return(tasa_continua_dias(spot,future,t))
        
#         else:
#             print("Options: mes, dias")
            
            
#     elif tipo.lower()== "tna":
        
#         if period.lower()=="mes":
#             return(tasa_tna_mes(spot,future,t))
        
#         elif period.lower()=="dias":
#             return(tasa_tna_dias(spot,future,t))
        
#         else:
#             print("Options: mes, dias")
        
        
#     elif tipo.lower()== "tea":
        
#         if period.lower()=="mes":
#             return(tasa_tea_mes(spot,future,t))
        
#         elif period.lower()=="dias":
#             return(tasa_tea_dias(spot,future,t))
        
#         else:
#             print("Options: mes, dias")

In [26]:
def tipo_tasa(spot,future,t=30,tipo="continua",period="dias"):
    
    if tipo.lower()== "continua":
        
        if period.lower()=="mes":
            return(tasa_continua_mes(spot,future,t))
        
        elif period.lower()=="dias":
            return(tasa_continua_dias(spot,future,t))
        
        else:
            print("Options: mes, dias")
            
            
    elif tipo.lower()== "tna":
        
        if period.lower()=="mes":
            return(tasa_tna_mes(spot,future,t))
        
        elif period.lower()=="dias":
            return(tasa_tna_dias(spot,future,t))
        
        else:
            print("Options: mes, dias")
        
        
    elif tipo.lower()== "tea":
        
        if period.lower()=="mes":
            return(tasa_tea_mes(spot,future,t))
        
        elif period.lower()=="dias":
            return(tasa_tea_dias(spot,future,t))
        
        else:
            print("Options: mes, dias")
            
    elif tipo.lower()== 'directa':
        return(tasa_directa(spot,future))
    
    else:
        return("Error")

In [27]:
spot_v=104.261841
future_v=105.28
dias=10
mes=(10/30)

tipo_tasa(spot_v,future_v)

0.11823603914081394

In [28]:
spot_v=104.261841
future_v=105.28
dias=10
mes=(10/30)

tipo_tasa(spot=spot_v,
          future=future_v,
          t=dias,
          tipo='directa',
          period='dias')

0.00976540400816428

### Days until settlement

Es importante tener en cuenta que al calcular las tasas de manera diaria se tienen que usar los dias exactos desde que se hace la transaccion hasta el dia del settlement. Generalmente el settlement se da el ultimo dia habil del mes de vencimiento de futuro, pero esto no siempre va a coincidir con el ultimo dia real del mes. Es por esto que tener una funcion que calcula los dias exactos hasta el ultimo dia habil del mes o settlement, es muy importante porque con este input se puede calcular la tasa correcta.

In [29]:
import calendar
from datetime import date

#Calculo el ultimo dia habil del mes, teniendo ano y mes como input.
#De la ultima semana del mes, toma el maximo dia de entre 0:5(lun-vier)
def last_business_day_in_month(year: int, month: int):
    return(date(year,month,max(calendar.monthcalendar(year, month)[-1:][0][:5])))

#Calculo el ultimo dia habil del mes, tomamos la ecuacion anterior
def days_until_last_business_day(year:int,month:int):
    
    delta = last_business_day_in_month(year,month) - date.today()
    return(delta.days)   

days_until_last_business_day(2022,2)

34

###  Bid - Offer Rates

Tomo las funciones creadas anteriormente para calcular las tasas tomadoras y colocadoras usando los datos del spot y futuros importados

In [30]:
#Final DataFrame
tasas_anuales_bid_offer =pd.DataFrame(columns=['bid','offer'])


#List variables
month_list= ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']
activo_tipo_list = ['DLR', 'GGAL','PAMP','YPFD']
activo_ano_list = ['22','23']
ano_list=[2022,2023]


# Function variables> elijo el tipo de tasa que se va a calcular
tipo='continua'
period='dias'


## Itero con las columnas'bid' y 'offer' del dataframe final
for column in tasas_anuales_bid_offer:
    ind=0
#     print("\n",column,"\n")
    
    ## Itero y tomo cada dato de 'bid' y 'offer', se usan los datos de los precios de los futuros
    for data in fut[column]:
#         print(f"data {data}")
        
        ## Itero para saber a que ano pertenece el futuro
        for ano_activo in activo_ano_list:
            
            ## Corroboro si el ano esta en el indice del precio a traer, y seteo el ano correspondiente
            ## para usar en la formula que calcula los dias hasta el settlement
            if ano_activo in fut[column].index[ind]:
                year=ano_list[activo_ano_list.index(ano_activo)]

#                 print(fut[column].index[ind])
                
                ## Corroboro el tipo de activo de donde viene la data, DLR,GGAL,PAMP o YPFD
                for tipo_activo in activo_tipo_list:

                    if tipo_activo in fut[column].index[ind]:
                        
                        ## Establezco el mes correspondiente a la data
                        for mes in month_list:


                            if mes in fut[column].index[ind].lower():
#                                 print(f"Mes: {mes}")

                                ## Teniendo el el precio del futuro,el ano y mes de settlement, y el tipo de activo 
                                ## del que quiero calcular la tasa, puedo traer el valor del spot, calcular los dias
                                ## hasta el settlement y calcular la tasa
            
                                #dataframe
                                index= fut[column].index[ind]
                                column= column

                                #function
                                spot= spot_df.iloc[-1][tipo_activo]
                                future= data
                                month = month_list.index(mes)+1


                                #output
                                tasas_anuales_bid_offer.loc[index,column] = round(tipo_tasa(spot=spot,
                                                                                      future=future,
                                                                                      t=days_until_last_business_day(year,month),
                                                                                      tipo=tipo,
                                                                                      period=period),6)
#                                 print(f"Tipo Activo: {tipo_activo}")
#                                 print(f"Spot: {spot}")
#                                 print(f"Future: {future}")
#                                 print(f"Year: {year}")
#                                 print(f"Month count: {month}")
#                                 print(f"Dias faltan: {days_until_last_business_day(year,month)}")
#                                 print(f"Tipo: {tipo}")
#                                 print(f"Periodo: {period}")
#                                 print(f"Tasa: {tasas_anuales_bid_offer.loc[index,column]}\n")

#             else:
#                 print(f"Other Year {ano_activo}")
                
#         print(f"Ind: {ind}\n")
        ind+=1

            
# tasas_anuales_bid_offer

Tasa colocadora: bid, Tasa tomadora: offer

In [31]:
tasas_anuales_bid_offer

,bid,offer
DLR/ENE22,0.313278,NaN
DLR/FEB22,0.408841,0.416742
DLR/MAR22,0.483768,0.491152
DLR/ABR22,0.488124,0.499566
DLR/MAY22,0.483777,0.497806
DLR/JUN22,0.49442,0.495325
DLR/JUL22,NaN,NaN
DLR/AGO22,NaN,NaN
DLR/SEP22,NaN,NaN
DLR/OCT22,NaN,NaN


In [32]:
tasas_anuales_bid_offer

,bid,offer
DLR/ENE22,0.313278,NaN
DLR/FEB22,0.408841,0.416742
DLR/MAR22,0.483768,0.491152
DLR/ABR22,0.488124,0.499566
DLR/MAY22,0.483777,0.497806
DLR/JUN22,0.49442,0.495325
DLR/JUL22,NaN,NaN
DLR/AGO22,NaN,NaN
DLR/SEP22,NaN,NaN
DLR/OCT22,NaN,NaN


In [33]:
tasas_anuales_bid_offer['arbitraje']= tasas_anuales_bid_offer['bid']>tasas_anuales_bid_offer['offer']
tasas_anuales_bid_offer

,bid,offer,arbitraje
DLR/ENE22,0.313278,NaN,False
DLR/FEB22,0.408841,0.416742,False
DLR/MAR22,0.483768,0.491152,False
DLR/ABR22,0.488124,0.499566,False
DLR/MAY22,0.483777,0.497806,False
DLR/JUN22,0.49442,0.495325,False
DLR/JUL22,NaN,NaN,False
DLR/AGO22,NaN,NaN,False
DLR/SEP22,NaN,NaN,False
DLR/OCT22,NaN,NaN,False


In [34]:
arbitraje_dict={}
count=0
for offer in tasas_anuales_bid_offer['offer']:
    for bid in tasas_anuales_bid_offer['bid']:
        if bid>offer:
            print(f"Bid: {tasas_anuales_bid_offer[tasas_anuales_bid_offer['bid'] == bid].index[0]} {bid},\nOffer: {tasas_anuales_bid_offer[tasas_anuales_bid_offer['offer'] == offer].index[0]} {offer}\n")
            arbitraje_dict.update({f"{tasas_anuales_bid_offer[tasas_anuales_bid_offer['bid'] == bid].index[0]}-{count}": (bid, (tasas_anuales_bid_offer[tasas_anuales_bid_offer['offer'] == offer].index[0], offer))})
            count+=1

Bid: DLR/MAR22 0.483768,
Offer: DLR/FEB22 0.416742

Bid: DLR/ABR22 0.488124,
Offer: DLR/FEB22 0.416742

Bid: DLR/MAY22 0.483777,
Offer: DLR/FEB22 0.416742

Bid: DLR/JUN22 0.49442,
Offer: DLR/FEB22 0.416742

Bid: DLR/JUN22 0.49442,
Offer: DLR/MAR22 0.491152

Bid: DLR/ENE22 0.313278,
Offer: PAMP/FEB22 0.266271

Bid: DLR/FEB22 0.408841,
Offer: PAMP/FEB22 0.266271

Bid: DLR/MAR22 0.483768,
Offer: PAMP/FEB22 0.266271

Bid: DLR/ABR22 0.488124,
Offer: PAMP/FEB22 0.266271

Bid: DLR/MAY22 0.483777,
Offer: PAMP/FEB22 0.266271

Bid: DLR/JUN22 0.49442,
Offer: PAMP/FEB22 0.266271



In [35]:
# arbitraje_dict

In [36]:
arbitraje_df = pd.DataFrame(arbitraje_dict).T.sort_index()
arbitraje_df.columns = ['bid','offer']
arbitraje_df

,bid,offer
DLR/ABR22-1,0.488124,"(DLR/FEB22, 0.416742)"
DLR/ABR22-8,0.488124,"(PAMP/FEB22, 0.266271)"
DLR/ENE22-5,0.313278,"(PAMP/FEB22, 0.266271)"
DLR/FEB22-6,0.408841,"(PAMP/FEB22, 0.266271)"
DLR/JUN22-10,0.49442,"(PAMP/FEB22, 0.266271)"
DLR/JUN22-3,0.49442,"(DLR/FEB22, 0.416742)"
DLR/JUN22-4,0.49442,"(DLR/MAR22, 0.491152)"
DLR/MAR22-0,0.483768,"(DLR/FEB22, 0.416742)"
DLR/MAR22-7,0.483768,"(PAMP/FEB22, 0.266271)"
DLR/MAY22-2,0.483777,"(DLR/FEB22, 0.416742)"


In [37]:
bid = 0.29502